In [14]:
%run binance_secrets.ipynb

In [19]:
import pandas as pd
from binance.client import Client


# Set up Binance API
client = Client(API_KEY, API_SECRET)



# Get historical price data for ETHUSDT from 2019-01-01 to 2021-01-01
symbol = 'ETHUSDT'
start_date = '2023-01-01'
end_date = '2023-06-01'
klines = client.get_historical_klines(symbol, Client.KLINE_INTERVAL_1MINUTE, start_date, end_date)

# Convert klines to DataFrame
columns = ['timestamp', 'open', 'high', 'low', 'close', 'volume', 'close_time', 'quote_asset_volume', 'num_trades', 'taker_buy_base', 'taker_buy_quote', 'ignore']
df = pd.DataFrame(klines, columns=columns)
df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
df.set_index('timestamp', inplace=True)

# Define trading parameters
trade_size = 1  # Adjust as needed
buy_momentum_threshold = 0.003
sell_momentum_threshold = -0.003
lookback_period = 60

# Initialize variables
portfolio_value = 10000  # Initial portfolio value
position = 0  # Number of assets held
trade_log = []  # List to store trade information

# Backtest loop
for i in range(lookback_period, len(df)):
    past_prices = df['close'][i - lookback_period:i]
    current_price = float(df['close'][i])
    price_change_percentage = (float(current_price) - float(past_prices.iloc[0])) / float(past_prices.iloc[0])
    
    if price_change_percentage > buy_momentum_threshold and position == 0:
        position = trade_size
        portfolio_value -= current_price * trade_size
        trade_log.append(('Buy', current_price))
    elif price_change_percentage < sell_momentum_threshold and position > 0:
        portfolio_value += current_price * position
        position = 0
        trade_log.append(('Sell', current_price))
    
# Calculate final portfolio value
final_portfolio_value = portfolio_value + (position * current_price)

# Print results
print("Initial Portfolio Value:", portfolio_value)
print("Final Portfolio Value:", final_portfolio_value)
print("Total Return:", (final_portfolio_value - portfolio_value) / portfolio_value * 100, "%")

open("trade_log.txt", "w").close()

log_file = open("trade_log.txt", "w")
for log in trade_log:
    log_file.write(f"{log[0]} : {log[1]}\n")
log_file.close()


Initial Portfolio Value: 8180.369999999997
Final Portfolio Value: 10055.179999999997
Total Return: 22.9184010014217 %
